In [ ]:
import pyalex
from pyalex import Authors
from pyalex import Works
import os
import itertools
import pandas as pd
import pickle

In [ ]:
# Give email to use polite pool, otherwise leave blank
pyalex.config.email = input()

In [ ]:
data_dir = "data/"
source_dir = "sources/"

if not os.path.exists(data_dir): os.makedirs(data_dir)
if not os.path.exists(source_dir): os.makedirs(source_dir)

## Download OA author ids using names

In [ ]:
authors_sources = pd.read_csv(source_dir + input() + ".csv")
authors_sources

In [ ]:
# Get lists of author objects grouped by display name
author_objects = []

for a in authors_sources['name']:
    print(a)
    author_objects.append( Authors().search_filter(display_name=a).get() )

In [ ]:
# Get author names and OA ids and save to list – need to check these by hand for relevance!

authors_ids = []

for author_list in authors_objects:   
    for author in author_list:
        authors_ids.append( (author['display_name'], author['id'], author['relevance_score'])  )
        
authors_ids_df = pd.DataFrame( authors_ids, columns=['name', 'id', 'relevance'] )

authors_ids_df

In [ ]:
# Save to csv for hand-checking
authors_ids_df.to_csv('sources/authors_ids_unchecked.csv')

## Download works using OA author ids

In [ ]:
authors_ids = pd.read_csv(source_dir + input() + ".csv")
authors_ids

In [ ]:
import time
authors_works = {}

for i, row in authors_ids.iterrows():
    
    author_name = row[0]
    author_id = row[1].replace('https://openalex.org/', '')
    
    print(i, author_name, 'https://openalex.org/' + author_id)
    
    paginator = Works().filter( authorships={"author" : {'id' : author_id}} ).paginate(per_page=200)
    works = list(itertools.chain.from_iterable(paginator))
    
    if author_name not in authors_works:
        authors_works[author_name] = {author_id : works}
    else:
        authors_works[author_name][author_id] = works
        
    time.sleep(10)

In [ ]:
print(len(authors_works.keys()))
print( authors_works.keys() )

In [ ]:
# Dump works as dict
with open('data/authors_works.p', 'wb') as fp:
    pickle.dump(authors_works, fp, protocol=pickle.HIGHEST_PROTOCOL)

## Get unique works by author as csv

In [ ]:
with open('data/authors_works.p', 'rb') as fp:
    authors_works = pickle.load(fp)

authors_works.keys()

## Get author references